<a href="https://colab.research.google.com/github/NBK-code/IMDB_Movie_Rating_Prediction/blob/main/IMDB_mrp_Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.svm import SVR 
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [2]:
X_train = pd.read_csv('/content/sample_data/X_train_scaled.csv')
X_val = pd.read_csv('/content/sample_data/X_val_scaled.csv')
X_test = pd.read_csv('/content/sample_data/X_test_scaled.csv')

In [3]:
y_train = pd.read_csv('/content/sample_data/y_train_scaled.csv')
y_val = pd.read_csv('/content/sample_data/y_val_scaled.csv')
y_test = pd.read_csv('/content/sample_data/y_test_scaled.csv')

In [5]:
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((3727, 43), (466, 43), (466, 43), (3727, 2), (466, 2), (466, 2))

In [29]:
X_train.head()

,Unnamed: 0,USA,UK,France,Canada,R,PG-13,PG,G,History,...,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,facenumber_in_poster,budget,actor_2_facebook_likes,movie_facebook_likes
0,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.498952,-0.913619,-0.520577,-0.824608,-1.963551,-1.720132,0.283456,-1.663723,-0.799471,0.590718
1,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.251562,-1.610674,2.235354,1.468370,2.521017,1.767974,0.283456,2.133861,2.128717,-1.051257
2,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.329754,0.170832,0.027888,0.987340,0.715569,-0.074789,0.860275,0.325623,1.772617,-1.051257
3,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.138383,0.546689,0.252142,1.354480,0.801017,2.382668,0.283456,0.818355,0.334583,1.415094
4,4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-1.534721,1.130255,-1.399331,-0.779355,-0.644141,0.991469,0.283456,-1.074267,-1.688927,-1.051257


In [30]:
y_train.head()

,Unnamed: 0,imdb_score
0,0,5.5
1,1,7.3
2,2,6.3
3,3,8.2
4,4,7.8


In [31]:
X_test.head()

,Unnamed: 0,USA,UK,France,Canada,R,PG-13,PG,G,History,...,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,facenumber_in_poster,budget,actor_2_facebook_likes,movie_facebook_likes
0,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.735613,-0.345177,0.252142,0.221824,-0.827408,-1.106576,-1.073757,0.559955,-0.060395,0.231648
1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-1.183926,0.467845,0.469015,-0.528296,-1.311966,-1.367711,0.860275,-1.663723,0.101769,0.314002
2,2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1.116986,0.442207,-0.939779,-0.470996,0.140587,0.613339,-1.073757,0.912601,-0.199589,1.249540
3,3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-0.498952,-0.087962,0.289574,-0.761191,-1.223630,-0.828640,0.283456,-0.144204,-0.131906,-1.051257
4,4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.189124,-0.264540,-1.260674,0.517600,0.060209,-0.630798,-1.073757,-0.144204,-0.803069,-1.051257


In [32]:
y_test.head()

,Unnamed: 0,imdb_score
0,0,4.5
1,1,6.8
2,2,5.7
3,3,2.8
4,4,6.7


In [33]:
X_val.head()

,Unnamed: 0,USA,UK,France,Canada,R,PG-13,PG,G,History,...,duration,director_facebook_likes,actor_3_facebook_likes,actor_1_facebook_likes,gross,num_voted_users,facenumber_in_poster,budget,actor_2_facebook_likes,movie_facebook_likes
0,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-0.859042,1.021217,0.198436,0.987340,0.060209,-1.030630,0.860275,-0.060363,0.290482,-1.051257
1,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.379516,0.224810,0.439787,-0.588979,-0.264671,-0.315336,0.283456,-0.189359,0.060977,-1.051257
2,2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.649124,-1.610674,0.607601,1.923073,-0.816859,-0.639693,0.860275,-0.189359,0.349171,-1.051257
3,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,-1.183926,-1.610674,-0.939779,-0.535972,-1.848496,-1.398692,-1.073757,0.016231,-1.004264,0.522649
4,4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.819149,1.189677,0.738932,1.176611,1.718805,1.583604,-1.073757,1.623839,0.836470,1.239672


In [34]:
y_val.head()

,Unnamed: 0,imdb_score
0,0,5.3
1,1,6.0
2,2,6.5
3,3,6.2
4,4,6.4


In [35]:
X_train.drop(columns=['Unnamed: 0'], inplace=True)
X_test.drop(columns=['Unnamed: 0'], inplace=True)
X_val.drop(columns=['Unnamed: 0'], inplace=True)

In [36]:
y_train.drop(columns=['Unnamed: 0'], inplace=True)
y_test.drop(columns=['Unnamed: 0'], inplace=True)
y_val.drop(columns=['Unnamed: 0'], inplace=True)

In [39]:
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
X_val_np = X_val.to_numpy()

In [44]:
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()
y_val_np = y_val.to_numpy()

In [50]:
X_train_np.shape, X_test_np.shape, X_val_np.shape

((3727, 42), (466, 42), (466, 42))

In [51]:
y_train_np.shape, y_test_np.shape, y_val_np.shape

((3727, 1), (466, 1), (466, 1))

In [52]:
y_train_np = y_train_np.flatten()
y_test_np = y_test_np.flatten()
y_val_np = y_val_np.flatten()

In [53]:
y_train_np.shape, y_test_np.shape, y_val_np.shape

((3727,), (466,), (466,))

##Random Forest

In [55]:
randregrssor=RandomForestRegressor(n_estimators=50)

In [56]:
randregrssor.fit(X_train_np,y_train_np)

RandomForestRegressor(n_estimators=50)

In [57]:
predictions = randregrssor.predict(X_test_np)

In [59]:
errors = abs(predictions - y_test_np)

In [60]:
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 0.56


In [61]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test_np)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 89.45 %.


##XGBoost

In [63]:
xgboost=XGBRegressor(n_estimators=50)

In [64]:
xgboost.fit(X_train_np,y_train_np)

[03:22:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(n_estimators=50)

In [65]:
predictions=xgboost.predict(X_test_np)

In [66]:
errors = abs(predictions - y_test_np)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test_np)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: 0.58 degrees.
Accuracy: 88.92 %.


##Decision Tree

In [73]:
dr=DecisionTreeRegressor()

In [74]:
dr.fit(X_train_np,y_train_np)

DecisionTreeRegressor()

In [75]:
predictions = dr.predict(X_val_np)

In [76]:
errors = abs(predictions - y_val_np)

In [77]:
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.73 degrees.


In [78]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_val_np)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 87.56 %.
